In [ ]:
from futu import *
import time
import pandas as pd

# 创建连接
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

# 创建过滤器列表
filter_list = []

# 1. 市值 > 100亿
market_val_filter = SimpleFilter()
market_val_filter.filter_min = 100 * 100000000  # 100亿
market_val_filter.filter_max = float('inf')
market_val_filter.stock_field = StockField.MARKET_VAL
market_val_filter.is_no_filter = False
market_val_filter.sort = SortDir.DESCEND  # 按市值降序排列
filter_list.append(market_val_filter)

# 2. PE TTM在8-20之间
pe_ttm_filter = SimpleFilter()
pe_ttm_filter.filter_min = 8
pe_ttm_filter.filter_max = 20
pe_ttm_filter.stock_field = StockField.PE_TTM
pe_ttm_filter.is_no_filter = False
filter_list.append(pe_ttm_filter)

# 3. 资产负债率 < 60%
debt_rate_filter = SimpleFilter()
debt_rate_filter.filter_min = 0
debt_rate_filter.filter_max = 60
debt_rate_filter.stock_field = StockField.DEBT_ASSET_RATE
debt_rate_filter.is_no_filter = False
filter_list.append(debt_rate_filter)

# 4. 净利率 > 10%
net_profit_rate_filter = SimpleFilter()
net_profit_rate_filter.filter_min = 10
net_profit_rate_filter.filter_max = float('inf')
net_profit_rate_filter.stock_field = StockField.NET_PROFIT_RATE
net_profit_rate_filter.is_no_filter = False
filter_list.append(net_profit_rate_filter)

# 5. ROE > 8%
roe_filter = SimpleFilter()
roe_filter.filter_min = 8
roe_filter.filter_max = float('inf')
roe_filter.stock_field = StockField.RETURN_ON_EQUITY_RATE
roe_filter.is_no_filter = False
filter_list.append(roe_filter)

# 6. 流动比率 > 1.5
current_ratio_filter = SimpleFilter()
current_ratio_filter.filter_min = 1.5
current_ratio_filter.filter_max = float('inf')
current_ratio_filter.stock_field = StockField.CURRENT_RATIO
current_ratio_filter.is_no_filter = False
filter_list.append(current_ratio_filter)

# 7. 振幅 < 3%
amplitude_filter = SimpleFilter()
amplitude_filter.filter_min = 0
amplitude_filter.filter_max = 3
amplitude_filter.stock_field = StockField.AMPLITUDE
amplitude_filter.is_no_filter = False
filter_list.append(amplitude_filter)

# 执行选股
nBegin = 0
last_page = False
all_stocks = []

try:
    while not last_page:
        ret, ls = quote_ctx.get_stock_filter(market=Market.HK, filter_list=filter_list, begin=nBegin, num=200)
        if ret == RET_OK:
            last_page, all_count, ret_list = ls
            print(f'获取结果: {len(ret_list)}个股票, 总计: {all_count}个')
            
            # 收集符合条件的股票
            for item in ret_list:
                stock_info = {
                    '股票代码': item.stock_code,
                    '股票名称': item.stock_name,
                    '最新价': item.cur_price,
                    '市值(亿)': round(item[market_val_filter] / 100000000, 2),
                    'PE TTM': round(item[pe_ttm_filter], 2),
                    '资产负债率(%)': round(item[debt_rate_filter], 2),
                    '净利率(%)': round(item[net_profit_rate_filter], 2),
                    'ROE(%)': round(item[roe_filter], 2),
                    '流动比率': round(item[current_ratio_filter], 2),
                    '振幅(%)': round(item[amplitude_filter], 2)
                }
                all_stocks.append(stock_info)
            
            # 更新偏移量
            nBegin += len(ret_list)
            
            # 避免触发频率限制
            if not last_page:
                time.sleep(3)
        else:
            print(f'查询失败: {ls}')
            break
    
    # 将结果转换为DataFrame并显示
    if all_stocks:
        result_df = pd.DataFrame(all_stocks)
        print("\n选股结果:")
        display(result_df)
    else:
        print("没有找到符合条件的股票")
        
except Exception as e:
    print(f"发生错误: {str(e)}")
    
finally:
    quote_ctx.close()  # 关闭连接

2025-05-18 16:33:35,382 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=780, host=127.0.0.1, port=11111, user_id=232724269
查询失败: 港股BMP权限下，不支持条件选股接口
没有找到符合条件的股票
查询失败: 港股BMP权限下，不支持条件选股接口
没有找到符合条件的股票


In [ ]:
from futu import *


2025-05-18 16:35:29,886 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=781, host=127.0.0.1, port=11111, user_id=232724269


In [25]:

quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, data = quote_ctx.get_plate_list(Market.HK, Plate.CONCEPT)
if ret == RET_OK:
    print(data)
    print(data['plate_name'][0])    # 取第一条的板块名称
    print(data['plate_name'].values.tolist())   # 转为 list
else:
    print('error:', data)
quote_ctx.close()  # 结束后记得关闭当条连接，防止连接条数用尽

2025-05-18 16:41:01,802 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=783, host=127.0.0.1, port=11111, user_id=232724269
             code   plate_name   plate_id
0     HK.LIST1110        阿里概念股   LIST1110
1     HK.LIST1161          双十一   LIST1161
2     HK.LIST1166         苹果概念   LIST1166
3     HK.LIST1175         一带一路   LIST1175
4     HK.LIST1176         5G概念   LIST1176
..            ...          ...        ...
101  HK.LIST22928     美国降息利好概念  LIST22928
102  HK.LIST23586  DeepSeek概念股  LIST23586
103  HK.LIST23589      智能驾驶概念股  LIST23589
104  HK.LIST23593       机器人概念股  LIST23593
105  HK.LIST23598      AI医疗概念股  LIST23598

[106 rows x 3 columns]
阿里概念股
['阿里概念股', '双十一', '苹果概念', '一带一路', '5G概念', '粤港澳大湾区', '特斯拉概念股', '啤酒', '体育用品', '稀土概念', '腾讯概念', '云办公', 'SaaS概念', '在线教育', '汽车经销商', '核电', '化妆品', '石油股', '电讯设备', '电力股', '手游股', '婴童用品股', '百货业股', '港口运输股', '电信股', '环保', '煤炭股', '综合车企股', '电池', '物流', '内地物业管理股', '农业股', '黄金股', '奢侈品品牌股', '电力设备股', '重型机械股', '食品股', '保险股', '纸业股

# 将全部data存到一个 csv 文件里
data.to_csv('hk_plate_list.csv', index=False, encoding='utf-8-sig')

低风险、稳定且高股息的股票板块：

​​电力股​​ (LIST1207)
公用事业板块，需求稳定，现金流可靠，通常有较高股息。

​​保险股​​ (LIST1228)
金融行业中的保险公司，盈利模式稳定，分红比例较高。

​​内银股​​ (LIST1239)
内地大型银行股，业务成熟，利息收入稳定，股息率较高。

​​水务股​​ (LIST1231)
公用事业类，需求刚性，现金流和分红政策稳定。

​​公路及铁路股​​ (LIST1246)
基础设施行业，收入受经济周期影响小，分红能力强。

​​高股息概念​​ (LIST1344)
直接定位为高股息的板块，符合筛选条件。

​​香港本地银行股​​ (LIST1252)
本地银行业务稳定，股息派发较为持续。

​​燃气股​​ (LIST1245)
公用事业属性强，市场需求稳定，分红稳健。

In [ ]:
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)

ret, dataLIST1228 = quote_ctx.get_plate_stock('HK.LIST1228')
if ret == RET_OK:
    print(dataLIST1228)
    print(dataLIST1228['stock_name'][0])    # 取第一条的股票名称
    print(dataLIST1228['stock_name'].values.tolist())   # 转为 list
    data.to_csv('​LIST1228.csv', index=False, encoding='utf-8-sig')
else:
    print('error:', dataLIST1228)
quote_ctx.close()  # 结束后记得关闭当条连接，防止连接条数用尽

2025-05-18 16:53:39,816 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=785, host=127.0.0.1, port=11111, user_id=232724269
        code  lot_size stock_name  stock_owner  stock_child_type stock_type  \
0   HK.00376      2000       云锋金融          NaN               NaN      STOCK   
1   HK.00945       100     宏利金融-S          NaN               NaN      STOCK   
2   HK.00966       200       中国太平          NaN               NaN      STOCK   
3   HK.01299       200       友邦保险          NaN               NaN      STOCK   
4   HK.01336       100       新华保险          NaN               NaN      STOCK   
5   HK.01339      1000   中国人民保险集团          NaN               NaN      STOCK   
6   HK.01508      1000      中国再保险          NaN               NaN      STOCK   
7   HK.02318       500       中国平安          NaN               NaN      STOCK   
8   HK.02328      2000       中国财险          NaN               NaN      STOCK   
9   HK.02378        50         保诚          NaN   

In [34]:

quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)  # 创建行情对象
snapshotDataLIST1228 = (quote_ctx.get_market_snapshot(
    dataLIST1228['code'].values.tolist()))  # 获取港股 HK.00700 的快照数据
quote_ctx.close()  # 关闭对象，防止连接条数用尽

2025-05-18 16:56:54,475 | 36521 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=786, host=127.0.0.1, port=11111, user_id=232724269


In [39]:
snapshotDataLIST1228[1].to_csv('LIST1228快照.csv', index=False, encoding='utf-8-sig')  # 保存快照数据到CSV文件